# SI649-21-FALL Lab 6 -> Altair III
School of Information, University of Michigan


## Assignment Overview
1. Interaction 
2. Review transform 
3. Comparing streamlit widget and altair widget

We will replicate 3 visualizations (and a bonus visualization) created by the article posted on [Five Thirty Eight](https://fivethirtyeight.com) available  [online](https://fivethirtyeight.com/features/the-mayweather-mcgregor-fight-as-told-through-emojis/) (Hickey, Koeze, Dottle, Wezerek 2017). 

**For this lab, please write Altair code to answer the questions. It's fine if your visualization looks slightly different from the example (e.g., getting 1.1 instead of 1.0, use orange instead of red, have different titles, chart width/height,and mark size/opacity)**

### Resources:
- Article by [Five Thirty Eight](https://fivethirtyeight.com) available  [online] (https://fivethirtyeight.com/features/the-mayweather-mcgregor-fight-as-told-through-emojis/) (Hickey, Koeze, Dottle, Wezerek 2017)  
- the original can be found on [Five Thirty Eight Mayweather vs McGregor] (https://github.com/fivethirtyeight/data/tree/master/mayweather-mcgregor)

### General Hints: 
* Yes, you can render emojis in colab/jupyter notebook and in your chart. You can consider them as text. 
* We recommend that you finish all the static charts before adding interactions. 
* If you see duplicated axes, use `axis=None` to get rid of unnecessary axes.  
* Don't forget to set `empty="none"`. The default behavior is that when nothing is selected, *everything* is selected. When set to none, empty selections contain no data values.  
* `resolve_scale` ensures charts share axes and scales. 


## Export to streamlit 

Like last lab, we want you to build a streamlit app for these visualization. You will be adding your visualizations to a streamlit .py file when you’re done. You can either do it as you go or at the end.

The app you build will have

1. A title with your name 
2. A selectbox in the sidebar that navigates to different charts. 
3. For chart 3, we want you to implement the radio widget interaction in streamlit, and putting BOTH the altair version and the streamlit version in the app. 
4. (OPTIONAL BONUS) For chart 4, we also want to see both the altair version and the streamlit version. 

Your finished app will look like this:

![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/uncompressed_full_export.gif)


In [1]:
# start with the setup
import pandas as pd
import altair as alt
import numpy as np
import pprint


ModuleNotFoundError: No module named 'altair'

In [ ]:
#load data 
df1=pd.read_csv("https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/data/df1.csv")
df2=pd.read_csv("https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/data/df2_count.csv")
df3=pd.read_csv("https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/data/df3.csv")
df4=pd.read_csv("https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/data/df4.csv")

In [ ]:
# change the 'datetime' column to be explicitly a datetime object
df2['datetime'] = pd.to_datetime(df2['datetime'])
df3['datetime'] = pd.to_datetime(df3['datetime'])
df4['datetime'] = pd.to_datetime(df4['datetime'])

## Visualization 1: Emoji and percentage of usage 

We will replicate the following visualization  
![vis1_static](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis1/static.png?raw=true)

**Description of the visualization (static):**
*   Use *df1* for this exercise
*   This visualization has 3 components: **emoji**, **percentage text**, and **bars** 
*   All 3 components share the same y axis, which display the *rank* of percentage from highest to lowest. 
*   The width of the bar(along x axis) encodes *PERCENT*
*   All 3 components have a low opacity because we want to add interactions (see the next cell). 


**Description of the visualization (interactivity):**
1. When hovering over bars, the associated emoji will show up as tooltip 
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis1/interactive_1.gif?raw=true)
2. When hovering over emojis, percentage texts, or bars, the opacity of the selected row will change to 1. 
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis1/interactive_2.gif?raw=true)
3. Brushing over the emojis will filter bars.
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis1/interactive_3.gif?raw=true)
4. Brushing over percentage text will filter bars.
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis1/interactive_4.gif?raw=true)



**Sample style settings (optional):**
Here's a list of default style settings we used to generate the graph.
* Original opacity for all 3 components: 0.6. 
* Hovered opacity: 1
* bar height = 15, color = orange
* text chart and emoji width are both 20
* after building the compound chart, use the following line to disable border : `.configure_view(strokeWidth=0)`



Hint:
* We recommend getting all static components working before writing any interactivity. 
* Add one interaction at a time and test whether or not it works. 
* To add an interaction that's not tooltip and zooming, you need four steps (review in-class demo). 
* Selection is used in two scenarios: 1) to add to a *condition*, which is used in `encode`. 2) to add in `transform_filter`. In this visualization, you will implement both. Think through which you will use where before trying to build this.

In [ ]:
df1.head()

In [ ]:
##TODO: replicate vis 1 
#Interaction requirement 2, change opacity when hover over 
hover_selection = alt.selection_single(on="mouseover")
brush_selection_emojis = alt.selection_interval(encodings=["y"])
brush_selection_text = alt.selection_interval(encodings=["y"])

opacityCondition_bar = alt.condition(hover_selection, alt.value(1), alt.value(0.6))
#Interaction requirement 3 and 4, create brushing filter  
##Static Component - Bars
bar = alt.Chart(df1).mark_bar(color="orange", opacity=0.6, height=15).encode(
    x=alt.X("PERCENT:Q", axis=None),
    y=alt.Y("EMOJI:N", axis=None, sort=alt.EncodingSortField(field="PERCENT", order="descending")),
    tooltip=alt.Tooltip("EMOJI", title=None),
    opacity=opacityCondition_bar
).transform_filter(brush_selection_emojis).transform_filter(brush_selection_text)
##Static Component - Emojis
emojis = alt.Chart(df1).mark_text(align="left", width=20, opacity=0.6).encode(
    text=alt.Text("EMOJI:N"),
    y=alt.Y("EMOJI:N", axis=None, sort=alt.EncodingSortField(field="PERCENT", order="descending")),
    opacity=opacityCondition_bar
).add_selection(brush_selection_emojis)
##Static Component - Text
text = alt.Chart(df1).mark_text(align="left", width=20, opacity=0.6).encode(
    text=alt.Text("PERCENT_TEXT:N"),
    y=alt.Y("EMOJI:N", axis=None, sort=alt.EncodingSortField(field="PERCENT", order="descending")),
    opacity=opacityCondition_bar
).add_selection(brush_selection_text)
##Put all together
(emojis | text | bar).add_selection(hover_selection).configure_view(strokeWidth=0).resolve_scale(y="shared")


## Visualization 2: Irish Pride vs. Money Team 

We will replicate the following visualization  
![vis2_static](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis2/static.png?raw=true)

**Description of the visualization (static):**
*   Use *df2* 
*   The visualization has 1 "static" component: **line chart**. It displays the relationship between *datetime* and *count* 


**Description of the visualization (interactivity):**
1. Enable zooming and panning along the x-axis. (The gif below only displays the line chart.)
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis2/interactive_1.gif?raw=true)
2. Display a vertical line that moves with the mouse. This will require you to add additional chart component(let's call it **vLine**).
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis2/interactive_2.gif?raw=true)
3. Display the intersection of the **vLine** with the **line chart** as 2 circles (let's call these two circles **intersection dots**). 
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis2/interactive_3.gif?raw=true)
4. When hovering over these **intersection dots**, display the *tweet_count*, *datetime*, and *team* in tooltip.  
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis2/interactive_4.gif?raw=true)



**Sample style settings (optional):**
Here's a list of default style settings we used to generate the graph.

* line chart size = 2.5, 
* vLine: size=4, color="lightgray", initial opacity = 0 
* indicator dot:  color="black" size=70

**Bugs**

If your interaction look similar to this, don't worry about it. It's likely a altair/colab issue. 

1. Blinking lines: You can fix by making your **vLine** thicker, or view it in vegalite editor. 

![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis2/interactive_2_bug1.gif?raw=true)


**Hint**


* We only want to enable zooming and panning along the x-axis.
*  There are multiple ways of implementing the **vLine**. Here is one of them: 
> 1) use mark_rule to generate a line for every single data point and set these line's opacity to be 0.

 > 2)when mouse hovering over a line, display it by changing its opacity. 

*  The implementation of the **intersection dots** is similar to that of the **vLine**. Do you need a new selection/condition for the **intersection dots**?




In [ ]:
#TODO: replicate vis2
#Zooming and Panning
line_selection = alt.selection_interval(bind="scales", encodings=["x"])
#vertical line
selection = alt.selection_single(on="mouseover", nearest=True, init={"x": 1503792330000.0})
opacity_condition = alt.condition(selection, alt.value(1), alt.value(0))
vLine = alt.Chart(df2).mark_rule(color="lightgray", size=4, opacity=0).encode(
    x=alt.X("datetime:T"),
    opacity=opacity_condition
).add_selection(selection)
#interaction dots
dots = alt.Chart(df2).mark_circle(color="black", size=70).encode(
    x=alt.X("datetime"),
    y=alt.Y("tweet_count:Q", title="Four-minute rolling average"),
    tooltip=["tweet_count:Q", "datetime:T", "team:N"],
    opacity=opacity_condition
).interactive(bind_y=False)
#Static component line chart
line = alt.Chart(df2).mark_line(size=2.5).encode(
    y=alt.Y("tweet_count:Q", scale=alt.Scale(domain=[0, 5.5]), title="Four-minute rolling average"),
    x=alt.X("datetime:T", title=None),
    color=alt.Color("team:N")
).add_selection(line_selection)
#Put all together
line + vLine + dots

## Visualization 3: Much hype, more boredom

We will replicate the following visualization  
![vis3_static](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis3/static.png?raw=true)

**Description of the visualization (static):**
*   Use *df3*
*   The visualization has 1 component: **line chart**. It displays the relationship between *datetime* and *tweet_count*. Each line represents one emoji. 

**Description of the visualization (interactivity):**
1. Build radio selections for emojis. Theoretically, only one line will be shown at any given time. See the "bug" section for more detail.
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis3/interactive_1.gif?raw=true)
2. Brushing over line chart will display individual data points as circles. This will require another chart component, let's call this component **circles**.  
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis3/interactive_2.gif?raw=true)



**Sample style settings (optional):**
Here's a list of default style settings we used to generate the graph.

* **circles**: color="black",opacity=0.7 
* For both x and y axis, we have the following tickCount specified
> axis=alt.Axis(tickCount=5,...) 



**Bugs**

If your interaction look similar to this, don't worry about it. It's likely a altair/colab issue. 

1. single click reset selection even when *clear=False*. This is likely a colab issue, because the chart will behave correctly in vega-editor. You can ignore this bug or view it in vegalite online editor. 

![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis3/interactive_2_bug.gif?raw=true)

**Hint**

* You will have 2 selections. One for the emoji selection and one for the time selection. Ensure that these two interactions work independently before merging them together. 

* You can use multiple transform_filter, or use logical operand to chain multiple selections. 

In [ ]:
df3

In [ ]:
##TODO: replicate vis3
emojis = sorted(set(df3["emoji"]))
widget = alt.binding_radio(options=emojis, name="Select Emoji", debounce=10)
line_selection = alt.selection_single(fields=["emoji"], init={"emoji": "🔥"}, bind=widget)
circle_selection = alt.selection_interval(init={"x":[0, 0], "y":[0, 0]}, empty="none")
line = alt.Chart(df3).mark_line().encode(
    x=alt.X("datetime:T", title=None, axis=alt.Axis(tickCount=5)),
    y=alt.Y("tweet_count:Q", title="Four-minute rolling average", axis=alt.Axis(tickCount=5)),
).add_selection(line_selection).transform_filter(line_selection)
circles = alt.Chart(df3).mark_circle(color="black").add_selection(circle_selection).encode(
    x=alt.X("datetime:T", title=None, axis=alt.Axis(tickCount=5)),
    y=alt.Y("tweet_count:Q", title="Four-minute rolling average", axis=alt.Axis(tickCount=5)),
    opacity=alt.condition(circle_selection, alt.value(0.7), alt.value(0)),
).add_selection(circle_selection).transform_filter(line_selection)
line + circles

**Comparing Altair Widget with Streamlit Widget**

* We want you to compare the altair radio selection with the streamlit radio select widget. 
* You should have two versions of vis 3 on your streamlit app to compare these. The first one will be using the altair radio selection that you have just created. The second one should use a streamlit widget to create radio buttons to switch between the two graphs.
* Put both of these visualizations on the Vis 3 page. See the gif below for an example of what both of these graphs should look like.

Your vis 3 export will look like this:
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis3/full_export.gif)

## BONUS: Visualization 4: Tears were shed-of joy and sorrow

OPTIONAL:
We will replicate the following visualization  
![vis4_static](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis4/static.png?raw=true)

**Description of the visualization (static):**
*   Use *df4*
*   The visualization has 2 components: **line chart** that displays relationship between *datetime* and *tweet_count*, and **legend** that displays the two emojis in the line chart.  
*   **legend** is a chart. It is not the automatically generated legend. (i.e., In the line chart's color encoding, set *legend=None*)
*   Two components are displayed side by side 

**Description of the visualization (interactivity):**
1. In the legend component, if one of the emoji is clicked, the selected emoji will have full opacity while the other emoji becomes transparent. 
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis4/interactive_1.gif?raw=true)
2.  Clicking emojis in the **legend** component will display the corresponding line and hide the other line . 
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis4/interactive_2.gif?raw=true)

**Sample style settings (optional):**
Here's a list of default style settings we used to generate the graph.
* For the line chart, we used tickCount=5 in x and y axis.  
* For the legend, we used size=25,strokeWidth=0

**Hint**

* You can have one selection and multiple conditions that use the same selection. 
* To hide a line, you have 2 options: changing opacity and adding transform_filter. In this chart, we want you to practice the 1st option (changing opacity). Using this method, your x- and y-axis will have a consistent scale.  

In [ ]:
# #TODO: BONUS: Replicate vis 4
emoji_selection = alt.selection_single(encodings=["y"])
emoji = alt.Chart(df4, title="legend", height=100).mark_text(size=25).encode(
    text=alt.Text("emoji:N"),
    y=alt.Y("emoji:N", axis=None, sort=["🤣", "😭"]),
    opacity=alt.condition(emoji_selection, alt.value(1), alt.value(0.005))
).add_selection(emoji_selection)
line = alt.Chart(df4, title="Tears were shed-of joy and sorrow").mark_line().encode(
    x=alt.X("datetime:T", title=None, axis=alt.Axis(tickCount=5)),
    y=alt.Y("tweet_count:Q", title="Four-minute rolling average", axis=alt.Axis(tickCount=5)),
    color=alt.Color("emoji:N", legend=None),
    opacity=alt.condition(emoji_selection, alt.value(1), alt.value(0))
)
(line | emoji).configure_view(strokeWidth=0).resolve_scale(y="shared")

**Comparing Altair Widget with Streamlit Widget**

* We want you to compare the altair radio selection with the streamlit radio select widget for vis 4 as well. 
* You should have two versions of vis 4 on your streamlit app to compare these. The first one will be using the altair radio selection that you have just created. The second one should use a streamlit widget to create radio buttons to switch between the two graphs.
* Put both of these visualizations on the Vis 3 page. See the gif below for an example of what both of these graphs should look like.

Your vis 4 export will look like this:
![alt text](https://raw.githubusercontent.com/eytanadar/si649public/master/lab6/hw/vis4/full_export.gif)

*This is the end of the lab*. 


Please run all cells (Runtime->Restart and run all), and 
1.  save to PDF (File->Print->Save PDF -> landscape, shrink to 80%)
2.  save to ipynb (File -> Download .ipynb)

Rename both files with your uniqname: e.g. uniqname.pdf/ uniqname.ipynb

Upload both files to canvas. 


